In [4]:
import csv

In [301]:
psv = open('test.psv')
dialect = csv.Sniffer().sniff(psv.read(1024))
reader = csv.DictReader(psv, dialect=dialect)
rows = []
for r in reader:
    rows += [r]

In [302]:
rows
copy = rows

In [303]:
df.tail(1)['date'].values[0]

'2000-05-18'

In [304]:
df['date'] = pd.date_range(df.head(1)['date'].values[0]
                ,df.tail(1)['date'].values[0]
                ,freq='D').to_series()

In [305]:
copy = [list(c.values()) for c in copy]

In [306]:
import datetime

df = pd.DataFrame(copy,columns=['date','pred_true'])

dayofweeks = []

for i,r in df.iterrows():
    dt = r['date']

    datestring = str(dt).split('-')
    year,month,day = int(datestring[0]), int(datestring[1]), int(datestring[2])
    
    dayofweek = datetime.datetime(year,month,day).weekday()
                     
    dayofweeks += [dayofweek]

In [307]:
df['dayofweek'] = weekdays

In [308]:
df['y_pred'] = df.pred_true.apply(lambda x:list(x)[0])
df['y_true'] = df.pred_true.apply(lambda x:list(x)[1])
df = df.drop('pred_true',axis=1)

In [310]:
df.head()

,date,dayofweek,y_pred,y_true
0,1999-12-31,4,0,0
1,2000-01-01,5,1,0
2,2000-01-02,6,0,1
3,2000-01-03,0,0,0
4,2000-01-04,1,0,1


$$recall = \frac{\;tp\;}{\;tp\;+\;fp\;}$$

$$precision = \frac{\;tp\;}{\; tp\; +\; fn \;}$$

$$f1 = 2 * \frac{precision * recall}{precision + recall}$$

In [322]:
def true_positives(pred,target,ds):
    
    return [dt for dt,p,t in zip(ds,pred.astype(int)
                    ,target.astype(int)) if p+t==2]


def false_positives(pred,target,ds):
    
    return [dt for dt,p,t in zip(ds,pred.astype(int)
                    ,target.astype(int)) if p+t==1 and p==1]


def false_negatives(pred,target,ds):
    
    return [dt for dt,p,t in zip(ds,pred.astype(int)
                    ,target.astype(int)) if p+1==1 and t==1]


def precision(pred,target,ds):
    
    tp = len(true_positives(pred,target,ds))
    fp = len(false_positives(pred,target,ds))
    
    return tp / (tp + fp)


def recall(pred,target,ds):

    tp = len(true_positives(pred,target,ds))
    fn = len(false_negatives(pred,target,ds))
    
    return tp / (tp + fn)


def f1(pred,target,ds):
    
    p = precision(pred,target,ds)
    r = recall(pred,target,ds)
    
    f1 = ( (r**-1 + p**-1) / 2 )**-1
    f1_ = 2 * ((p * r) / (p + r))
    
    return f1

In [323]:
print('global recall: ',recall(df.y_pred,df.y_true,df.date))
print('gloabl precision: ',precision(df.y_pred,df.y_true,df.date))
print('global f1: ',f1(df.y_pred,df.y_true,df.date))

global recall:  0.46153846153846156
gloabl precision:  0.47619047619047616
global f1:  0.46875


In [324]:
tuesday_df = df[df.dayofweek==3]
tuesday_df.shape[0] - df.shape[0]

-120

In [325]:
print('Tuesday recall: ',recall(tuesday_df.y_pred,tuesday_df.y_true,tuesday_df.date))
print('Tuesday precision: ',precision(tuesday_df.y_pred,tuesday_df.y_true,tuesday_df.date))
print('Tuesday f1: ',f1(tuesday_df.y_pred,tuesday_df.y_true,tuesday_df.date))

Tuesday recall:  0.4
Tuesday precision:  0.5714285714285714
Tuesday f1:  0.47058823529411764


In [326]:
print('Tuesday f1: ',f1(tuesday_df.y_true,tuesday_df.y_pred,tuesday_df.date))

Tuesday f1:  0.47058823529411764


In [327]:
print('Tuesday precision: ',precision(tuesday_df.y_true,tuesday_df.y_pred,tuesday_df.date))

Tuesday precision:  0.4
